# Car Classification

In [3]:
# Install
print("📦 Installing dependencies...")
!pip install -q flask tensorflow keras pillow flask-cloudflared
!pip install flask-cors

# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# Setup paths
import os, shutil
DRIVE_PATH = '/content/drive/MyDrive/Colab Notebooks/Data Mining/my_model.h5'
LOCAL_PATH = '/content/my_model.h5'

if os.path.exists(DRIVE_PATH):
    print(f"✅ Copying model ({os.path.getsize(DRIVE_PATH)/(1024*1024):.1f} MB)...")
    shutil.copy(DRIVE_PATH, LOCAL_PATH)
    print("✅ Model ready!")

# Import & Setup
from flask import Flask, request, jsonify
from flask_cloudflared import run_with_cloudflared
from werkzeug.utils import secure_filename
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import numpy as np
from flask_cors import CORS

app = Flask(__name__)
CORS(app)
run_with_cloudflared(app)

# Load model
model = tf.keras.models.load_model(LOCAL_PATH)
UPLOAD_FOLDER = '/content/data_tes'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

# Routes
@app.route('/')
def index():
    return "<h1>✅ Car Classification API Running!</h1><p>POST to /predict2</p>"

@app.route('/predict2', methods=['POST'])
def predict():
    try:
        file = request.files['file']
        filename = secure_filename(file.filename)
        filepath = os.path.join(UPLOAD_FOLDER, filename)
        file.save(filepath)

        img = load_img(filepath, target_size=(150, 150))
        x = img_to_array(img) / 255.0
        x = x.reshape((1,) + x.shape)

        probs = model.predict(x, verbose=0)[0]
        label = int(np.argmax(probs))

        os.remove(filepath)

        return jsonify({
            "success": True,
            "Bus": float(probs[0]),
            "PickUp": float(probs[1]),
            "Sport": float(probs[2]),
            "label": label,
            "predicted_class": ['Bus', 'PickUp', 'Sport'][label]
        })
    except Exception as e:
        return jsonify({"success": False, "error": str(e)}), 500

# Run
print("\n" + "="*60)
print("🚀 STARTING SERVER")
print("="*60)
print("\n⏳ Please wait for Cloudflare URL...\n")

app.run()

📦 Installing dependencies...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Copying model (108.7 MB)...
✅ Model ready!



🚀 STARTING SERVER

⏳ Please wait for Cloudflare URL...

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on https://unavailable-waiting-lib-liberal.trycloudflare.com
 * Traffic stats available on http://127.0.0.1:8898/metrics
